   *«В математическом анализе всё должно быть точно. Какой смысл в неточном утверждении? Вот у Вас есть номер телефона. Какой в нём смысл, если Вы знаете его неточно?»*

   -- Дмитрий Беклемишев

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании «Хоть потоп». В данной работе будет разработан метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

<div class="alert alert-info">
<b>Комментарий:</b>
То что сделано в данном проекте является одним из самых странных и непрактичных решений, которое я видел в своей жизни, хотя видел я не так уж и много. На мой взгляд подобныее задачи лучше решать другими средствами. Например, хранить указанные данные в зашифрованом архиве и/или в зашифрованом разделе диска.

Тем не менее всё это представляет занятную тонкость, однако я не могу представить ситуации, где она действительно могла бы пригодиться.
</div>

## Загрузка данных

In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

Посмотрим на загружаемые данные.

In [3]:
data.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Для удобства переименуем названия столбцов.

In [4]:
data = data.rename(columns={'Пол': 'sex',
                            'Возраст': 'age',
                            'Зарплата' : 'cash',
                            'Члены семьи' : 'family',
                            'Страховые выплаты' : 'count'
                           })

И изменим тип данных в столбце о возрасте на более подходящий.

In [5]:
data['age'] = data['age'].astype(int)

В указанных данных нет явных пропусков. Поиск дубликатов затруднён форматом указанных данных, поэтому будем считать, что в указанных данных их нет.

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sex     5000 non-null   int64  
 1   age     5000 non-null   int64  
 2   cash    5000 non-null   float64
 3   family  5000 non-null   int64  
 4   count   5000 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 195.4 KB


## Умножение матриц

Ответим на вопрос о качестве линейной регрессии при умножении матрицы признаков на матрицу, имеющую обратную матрицу.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P^{-1}$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

---

**Ответ:** В обеих задачах параметры линейной регрессии определяются на основе матрицы признаков $X$ и вектора целевого признака $\vec{y}$. Если мы умножим исходную матрицу признаков на обратимую матрицу, то такая формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Превратиться в следующуюю:

$$
w' = ((XP^{-1})^T XP^{-1})^{-1} (XP^{-1})^T y
$$

Далее, применяя, свойства обратой матрицы можем получить:

$$
w' = ((P^{-1})^T X^T X P^{-1})^{-1} (P^{-1})^T X^T y
$$

$$
w' = P (X^T X)^{-1} P^T (P^{-1})^T X^T y
$$

$$
w' = P (X^T X)^{-1} (E)^T X^T y
$$

$$
w' = P (X^T X)^{-1} (E X)^T y
$$

$$
w' = P (X^T X)^{-1} (X)^T y
$$

Учитывая выражение для $w$:

$$
w' = (P^{-1})^{-1} w
$$

Таким образом задача обучения для $w'$ может быть поставлена следующим образом:

$$
w' = \arg\min_w MSE(XP^{-1}w, y)
$$

Для того, чтобы из новой задачи получить старые предсказания ($a = a'$), можно прибегнуть к следующему выражению.

$$
a' = Xw' = X(P^{-1})^{-1} w
$$

## Алгоритм преобразования

Для востановления персональной информации обучим модель, которая бы могла востанавливать её по остальным данным. Для этого разделим исходные данные на неперсональные данные, по которым мы будем обучать модель и персональные данные, которые мы будем восстанавливать.

In [7]:
features = data.drop('count', axis=1)
target = data['count']

Для восстановления будем применять модель линейной регрессии. Найдём среднеквадратичную ошибку такой модели на указанных данных.

In [8]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return np.dot(test_features,self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550286668


Для большей сложности восстановления введём ключ -- квадратную матрицу, подставляемую в задачу обучения таким образом, чтобы из $w$ получить $w'$, а именно, умножив $X$ на эту матрицу квадратную матрицу на этапе обучения с последующем домножением этой квадратной матрицы на $w$ этапе предсказаний.

In [9]:
rrrr = 42 # фиксируем последовательность
random.seed(rrrr)

s = len(data.columns)
key = np.random.rand(s,s)
while np.linalg.det(key) == 0:
    key = np.random.rand(s,s)

key

array([[0.00988591, 0.91403186, 0.36156845, 0.88658255, 0.28022072],
       [0.45284271, 0.66672081, 0.92205821, 0.0555644 , 0.33581081],
       [0.055686  , 0.91208412, 0.79756448, 0.80432457, 0.19752541],
       [0.80854408, 0.77053618, 0.0726572 , 0.14558914, 0.87222955],
       [0.07675279, 0.1892761 , 0.99087406, 0.97346773, 0.55246787]])

Новый метод даёт тоже самое среднее квадратичное отклонение, что и старый, без ключа.

In [10]:
class KeyLinearRegression:
    def fit(self, train_features, train_target, key):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        X = np.dot(X,key)
        y = train_target
        self.w = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y)

    def predict(self, test_features, key):
        ww = np.dot(key,self.w)
        w = ww[1:]
        w0 = ww[0]
        return np.dot(test_features,w) + w0
    
model = KeyLinearRegression()
model.fit(features, target, key)
predictions = model.predict(features, key)
print(r2_score(target, predictions))

0.4249391124904376


При этом, если попытаться подставить в этап предсказаний неправильный ключ, то можно получить совершенно неожиданный результаты.

In [11]:
rrrr = 14 # фиксируем последовательность
random.seed(rrrr)

s = len(data.columns)
nekey = np.random.rand(s,s)
while np.linalg.det(nekey) == 0:
    nekey = np.random.rand(s,s)

nekey

array([[0.49749235, 0.53833892, 0.29968431, 0.53741985, 0.90438962],
       [0.81011997, 0.31524245, 0.33408013, 0.47968869, 0.52124193],
       [0.76169138, 0.1724569 , 0.23677016, 0.9391132 , 0.78519702],
       [0.03532556, 0.00880531, 0.77336012, 0.30141145, 0.3542815 ],
       [0.18297066, 0.2156985 , 0.80136748, 0.1712449 , 0.99835233]])

In [12]:
model = KeyLinearRegression()
model.fit(features, target, key)
predictions = model.predict(features, nekey)
print(r2_score(target, predictions))

-61174831784.225784


## Вывод

В данном анализе был предложен механизм шифрования персональной информации клиентов страховой компании. Данный метод позволит не только скрыть персональную информацию, но и остальные столбцы в указанных данных, если применять его к таковым. Стоит отметить, что данный алгоритм будет успешно работать только в том случае, если всместо указанных фиксированных последовательностей использовать более длинные. В противном случае ключи можно легко подобрать.